# Restart Field

In this turorial Deepfield we deomnstrate processing of restart models. Recall that restart models is a special option used in simulators to continue modeling from the moment the calculation of the previous model is finished. The previous model is called the histrorical model.

In [1]:
import sys
sys.path.append('..')

from deepfield import Field, RestartField, execute_tnav_models

First, let us use tNavigator to calculate both historical and restart models using `execute_tnav_models` function

In [2]:
tnav_binary_path =  # path to tNav binary
tnav_licence_url =  # licence url

Calculate the historical model:

In [3]:
execute_tnav_models('../bin/tnav_run.sh',
                    ['../open_data/norne_simplified/norne_simplified.data'],
                    tnav_licence_url,
                    tnav_path=tnav_binary_path,
                    logfile='run.log')

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [04:30<00:00, 270.67s/it]


Calculate the restart model:

In [5]:
execute_tnav_models('../bin/tnav_run.sh',
                    ['../open_data/norne_simplified_restart/norne_simplified_restart.data'],
                    tnav_licence_url ,
                    tnav_path=tnav_binary_path,
                    logfile='run_restart.log')

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:52<00:00, 172.26s/it]


The restart model is loaded using `RestartField`:

In [7]:
field = RestartField('../open_data/norne_simplified_restart/norne_simplified_restart.data').load()

INFO:Field:Start reading norne_simplified_restart.data
INFO:Field:[norne_simplified_restart.data:1] Loading RESTARTDATE
INFO:Field:[norne_simplified_restart.data:21] Include .../open_data/norne_simplified_restart/schedule.inc
INFO:Field:Start reading schedule.inc
INFO:Field:[schedule.inc:5] Loading DATES
INFO:Field:[schedule.inc:27] Loading DATES
INFO:Field:[schedule.inc:49] Loading DATES
INFO:Field:[schedule.inc:71] Loading DATES
INFO:Field:[schedule.inc:91] Loading DATES
INFO:Field:[schedule.inc:112] Loading DATES
...............................................................................................................................
INFO:Field:Start reading ../open_data/norne_simplified_restart/RESULTS/norne_simplified_restart.RSM
INFO:Field:Finish reading ../open_data/norne_simplified_restart/RESULTS/norne_simplified_restart.RSM
INFO:Field:===== Field summary =====
ERROR:Field:Start date and DATES are not monotone.
INFO:Field:STATES attributes: PRESSURE, RS, SGAS, SOIL, SWAT


The loaded `field` contains data for both the historical model and the restart model. The historical model can be accessed using the `field.parent` attribute, while the data for the restart model is stored directly in `field`.

Some componets are shared between both models, for example, grid data:

In [12]:
field.grid is field.parent.grid

True

In contrast, the production rates in the two models are different because they correspond to different time periods. For example, let us compare the production rates in the historical model:

In [6]:
field.parent.wells['D-3AH'].results

,DATE,WWPR,WOPR,WGPR,WWIR,WBHP
0,1997-11-06,0.0,0.0,0.0,0.0,0.0000
1,1997-11-14,0.0,0.0,0.0,0.0,0.0000
2,1997-12-01,0.0,0.0,0.0,0.0,0.0000
3,1997-12-17,0.0,0.0,0.0,0.0,0.0000
4,1998-01-01,0.0,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...
243,2006-11-01,0.0,0.0,0.0,0.0,197.4303
244,2006-11-09,0.0,0.0,0.0,0.0,198.2078
245,2006-11-11,0.0,0.0,0.0,0.0,198.2498
246,2006-11-17,0.0,0.0,0.0,0.0,197.9522


with production rates in the restart model:

In [7]:
field.wells['D-3AH'].results

,DATE,WWPR,WOPR,WGPR,WWIR,WBHP
0,2003-01-01,0.0,0.0,0.0,0.0,210.0209
1,2003-01-02,0.0,0.0,0.0,0.0,210.0584
2,2003-01-12,0.0,0.0,0.0,0.0,209.8909
3,2003-01-20,0.0,0.0,0.0,0.0,209.9832
4,2003-02-01,0.0,0.0,0.0,0.0,209.8230
...,...,...,...,...,...,...
96,2006-11-01,0.0,0.0,0.0,0.0,197.2492
97,2006-11-09,0.0,0.0,0.0,0.0,198.0447
98,2006-11-11,0.0,0.0,0.0,0.0,198.0919
99,2006-11-17,0.0,0.0,0.0,0.0,197.8105


The historical and restart models can be concatenated in a single model:

In [11]:
full_field = field.full_model()

The concatenated model covers both historical and restart time periods, e.g.:

In [12]:
full_field.wells['D-3AH'].results

,DATE,WWPR,WOPR,WGPR,WWIR,WBHP
0,1997-11-06,0.0,0.0,0.0,0.0,0.0000
1,1997-11-14,0.0,0.0,0.0,0.0,0.0000
2,1997-12-01,0.0,0.0,0.0,0.0,0.0000
3,1997-12-17,0.0,0.0,0.0,0.0,0.0000
4,1998-01-01,0.0,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...
243,2006-11-01,0.0,0.0,0.0,0.0,197.2492
244,2006-11-09,0.0,0.0,0.0,0.0,198.0447
245,2006-11-11,0.0,0.0,0.0,0.0,198.0919
246,2006-11-17,0.0,0.0,0.0,0.0,197.8105


Done!